# Python操作JSON出现乱码的解决方案
> 其实刚刚写过一整篇Python编码问题的解决方案，由于JSON又是一种特殊案例（与库相关，与语言本身无关）所以就单独提出来说。


## 我们来看一个从网上获取json并又存到本地文件的例子

In [5]:
import requests,json

r = requests.get('https://api.github.com/repos/solomonxie/\
solomonxie.github.io/issues/25/comments')

# 获取到我的github中某条issue的所有评论，形式为<JSON格式的字符串>
comments = json.loads( r.content )

# 取某一条评论查看内容（中文）
cc = comments[0]['body'][0:10] # 取出的内容是'## 配置：先从配置'

In [6]:
cc, type(cc)

# 可以看到输出的是<unicode格式的字符串> 注：这时如果用print是能正常出中文的

(u'## \u914d\u7f6e\uff1a\u5148\u4ece\u914d\u7f6e', unicode)

### 好，到这里先停一下！
JSON的读取到目前为止，都是正常的：JSON Object对象给出的值都是unicode，没有被莫名转义，也没有报错误。
> 但是，unicode格式，意味着它和str格式不兼容！
这时，害羞的大姑娘Unicode刚出炉，你不能在这个时候让它和Str操作在一起！
报错也往往就在这种疏于防备的时候！

比如你看：

In [10]:
cc.encode('utf-8') + '显性声明的字符串'

'## \xe9\x85\x8d\xe7\xbd\xae\xef\xbc\x9a\xe5\x85\x88\xe4\xbb\x8e\xe9\x85\x8d\xe7\xbd\xae\xe6\x98\xbe\xe6\x80\xa7\xe5\xa3\xb0\xe6\x98\x8e\xe7\x9a\x84\xe5\xad\x97\xe7\xac\xa6\xe4\xb8\xb2'

In [11]:
cc + '显性声明的字符串'.decode('utf-8')

u'## \u914d\u7f6e\uff1a\u5148\u4ece\u914d\u7f6e\u663e\u6027\u58f0\u660e\u7684\u5b57\u7b26\u4e32'

In [12]:
cc + '显性声明的字符串'

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe6 in position 0: ordinal not in range(128)

上面打印了三条Unicode和Str的结合，
前两条分别是以Str格式的结合，以Unicode格式的结合。
但是第三条，把两个不同格式的字符串结合，就出错了。

对不起，这里不是Javascript，变量不可以任意交合。Python对变量和编码都是极其谨慎的。

所以明白了这点，我们再继续。

### 上面获得了JSON Object对象，那么再来试试将`JSON对象`整体存到文本文件中。
如果要存到本地文件，那么就必须把Object对象转换为Str格式的字符串。
json库自带.dumps()函数可以进行转化。
但是这里问题出现了！我们来小试一下：

In [370]:
print json.dumps( {"body": "你好"} )

 {"body": "\u4f60\u597d"}


> 竟然连`print大法`都不能把`json.dumps()`返回的内容正确打印出来。经过各种测试和查看官网对于此函数的文档，发现：

### 原来`json.dumps()`是默认所有非ascii码强制转化为代号（而非汉字）的，于`repr()`效果等同！
[官方文档](https://docs.python.org/2/library/json.html#encoders-and-decoders)里有说明，`json.dumps()`里面有个`ensure_ascii`参数，默认为True。
意思就是默认把所有非ascii字码用`\`强制转化。所以，为了关闭这个功能，我们必须把它设为`False`.
下面是个小测试：

In [372]:
json.dumps({"body": "你好"}, ensure_ascii=False)

'{"body": "\xe4\xbd\xa0\xe5\xa5\xbd"}'

### 这样一来JSON在Python里的编码问题就解决了：须用`json.dumps(obj,  ensure_ascii=False)`来转化为字符串

下面是完整的代码测试：

In [1]:
# @网络资源到本地存储真实测试
import requests,json

r = requests.get('https://api.github.com/repos/solomonxie/solomonxie.github.io/issues/25/comments')

# 获取到我的github中某条issue的所有评论，形式为<JSON格式的字符串>
comments = json.loads( r.content )

outgoing = json.dumps( comments, ensure_ascii=False )

with open('test.txt', 'w') as f:
    f.write(outgoing.encode('utf-8'))
with open('test.txt', 'r') as f:
    read = f.read()

In [3]:
read[0:20], type(read)

('[{"body": "## \xe9\x85\x8d\xe7\xbd\xae', str)

In [2]:
print read[0:20], type(read)

[{"body": "## 配置 <type 'str'>


## 大功告成！